In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%pip install whitebox
%pip install rasterio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#librerias a usar
from bs4 import BeautifulSoup  #leer archivos html
import json                    #manejar archivos json
import re                      #manejar funciones regulares
import shutil                  #manejo de archivos
import rasterio                #manejo de archivos geologicos
import os
import pkg_resources
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import imageio
import cv2
%matplotlib inline

import whitebox
wbt = whitebox.WhiteboxTools()

In [ ]:
#directorio
data_dir = '/content/drive/MyDrive/CurvasHipsometricas2/CUENCAS VERSION 2/cuenca_raster'        #definimos la carpeta donde se encuentra
n = 4                     #numero de caracteres de la execion del archivo contando el punto

output = "/content/drive/MyDrive/CurvasHipsometricas2/CUENCAS VERSION 2/RESULTADOS_SUBCUENCAS"
dir = output


#procesar varios archivos con el mismo nombre que cambia de numeros
Nombre_archivo = 'cuenca_'       #nombre base de todos los archivos
file = []
NumeroImagenes = 2117
ini = 0                  #el numero de inicio de las cuencas

for i in range(ini, NumeroImagenes + 1):
    file.append(Nombre_archivo + str(ini) + '.tif')
    ini = ini + 1

print(file)


['cuenca_0.tif', 'cuenca_1.tif', 'cuenca_2.tif', 'cuenca_3.tif', 'cuenca_4.tif', 'cuenca_5.tif', 'cuenca_6.tif', 'cuenca_7.tif', 'cuenca_8.tif', 'cuenca_9.tif', 'cuenca_10.tif', 'cuenca_11.tif', 'cuenca_12.tif', 'cuenca_13.tif', 'cuenca_14.tif', 'cuenca_15.tif', 'cuenca_16.tif', 'cuenca_17.tif', 'cuenca_18.tif', 'cuenca_19.tif', 'cuenca_20.tif', 'cuenca_21.tif', 'cuenca_22.tif', 'cuenca_23.tif', 'cuenca_24.tif', 'cuenca_25.tif', 'cuenca_26.tif', 'cuenca_27.tif', 'cuenca_28.tif', 'cuenca_29.tif', 'cuenca_30.tif', 'cuenca_31.tif', 'cuenca_32.tif', 'cuenca_33.tif', 'cuenca_34.tif', 'cuenca_35.tif', 'cuenca_36.tif', 'cuenca_37.tif', 'cuenca_38.tif', 'cuenca_39.tif', 'cuenca_40.tif', 'cuenca_41.tif', 'cuenca_42.tif', 'cuenca_43.tif', 'cuenca_44.tif', 'cuenca_45.tif', 'cuenca_46.tif', 'cuenca_47.tif', 'cuenca_48.tif', 'cuenca_49.tif', 'cuenca_50.tif', 'cuenca_51.tif', 'cuenca_52.tif', 'cuenca_53.tif', 'cuenca_54.tif', 'cuenca_55.tif', 'cuenca_56.tif', 'cuenca_57.tif', 'cuenca_58.tif', 'cuenc

In [ ]:
#definir dataset
def dataset(data_dir):

    wbt.set_working_dir(data_dir)    #indicamos que esta va a ser el dataset
    wbt.verbose = False              #desactivamos notificaciones

    print(os.listdir(data_dir))      #imprimimos los archivos

def print_raster(file,n):
    filename = str(name(file,n))
    filename = (filename + '.png')
    print(filename)
    # print the raster
    raster = imageio.imread(os.path.join(data_dir, file))
    raster1=np.where(raster==-99999.,np.nan,raster) #99999
    plt.imshow(raster1)
    plt.colorbar()
    plt.savefig(filename)
    plt.show()

def move(inicio):
    inicio = str(data_dir + "/" + inicio)
    #print(inicio)
    shutil.move(inicio, output)

def name(namefile, n):
    name = namefile
    n = n

    name = name[:-n]
    return(name)

def hillshade(file,n):
    filename = str(name(file,n))
    filename = (filename + '_hillshade.tif')

    wbt.hillshade(
        file,
        filename,
        azimuth=315.0,
        altitude=30.0,
        zfactor=None
    )

    move(filename)

def slope(file,n):
    filename = str(name(file,n))
    filename = (filename + '_slope.tif')

    wbt.slope(
    file,
    filename,
    zfactor=None,
    units="degrees", # 'degrees', 'radians', 'percent'
    )

    move(filename)

def aspect(file,n):
    filename = str(name(file,n))
    filename = (filename + '_aspect.tif')

    wbt.aspect(
    file,
    filename,
    zfactor=None,
    )

    move(filename)


def plan_curvature(file,n):
    filename = str(name(file,n))
    filename = (filename + '_plan_curvature.tif')

    # plan curvature
    wbt.plan_curvature(
    file,
    filename,
    log=False, #Display output values using a log-scale
    zfactor=None,
    )

    move(filename)

def profile_curvature(file,n):
    filename = str(name(file,n))
    filename = (filename + '_profile_curvature.tif')

    #profile curvature
    wbt.profile_curvature(
    file,
    filename,
    log=False,
    zfactor=None,
    )

    move(filename)

def total_curvature(file,n):
    filename = str(name(file,n))
    filename = (filename + '_total_curvature.tif')

    #total curvature Total curvature measures the curvature of the topographic surface rather than the curvature of a line across the surface in some direction
    wbt.total_curvature(
    file,
    filename,
    log=False,
    zfactor=None,
    )

    move(filename)


def average_normal_vector_angular_deviation(file,n):
    filename = str(name(file,n))
    filename = (filename + '_Roughness.tif')

    # Roughness
    wbt.average_normal_vector_angular_deviation(
    file,
    filename,
    filter=11, #Size of the filter kernel
    )

    move(filename)

def roughness_index(file,n):
    filename = str(name(file,n))
    filename = (filename + '_roughness_index.tif')

    # roughness index calculates the root-mean-square-deviation (RMSD) for each grid cell in a digital elevation model (DEM), calculating the residuals (i.e. elevation differences) between a grid cell and its eight neighbours.
    wbt.ruggedness_index(
    file,
    filename,
    )

    move(filename)

def circular_variance_of_aspect(file,n):
    filename = str(name(file,n))
    filename = (filename + '_aspect_variance.tif')

    # circular variance of aspect
    wbt.circular_variance_of_aspect(
    file,
    filename,
    filter=11,  #Size of the filter kernel
    )

    move(filename)


def dev_from_mean_elev(file,n):
    filename = str(name(file,n))
    filename = (filename + '_dev_elev.tif')

    #difference between the elevation of each grid cell and the mean elevation of the centering local neighbourhood, normalized by standard deviation
    wbt.dev_from_mean_elev(
    file,
    filename,
    filterx=11,
    filtery=11,
    )

    move(filename)



def diff_from_mean_elev(file,n):
    filename = str(name(file,n))
    filename = (filename + '_diff_elev.tif')

    # the difference between the elevation of each grid cell and the mean elevation of the centering local neighbourhood
    wbt.diff_from_mean_elev(
    file,
    filename,
    filterx=11,
    filtery=11,
    )

    move(filename)


def edge_density(file,n):
    filename = str(name(file,n))
    filename = (filename + '_edge_density.tif')

    # Edge density
    wbt.edge_density(
    file,
    filename,
    filter=11,
    norm_diff=5.0,
    zfactor=None,
    )

    move(filename)


def find_ridges(file,n):
    filename = str(name(file,n))
    filename = (filename + '_ridges.tif')

    # find ridges
    wbt.find_ridges(
    file,
    filename,
    line_thin=True,
    )

    move(filename)


def geomorphons(file,n):
    filename = str(name(file,n))
    filename = (filename + '_geomorphons.tif')

    #geomorphons landform classification. Jasiewicz et al. (2013) identified 10 common landform types by reclassifying related geomorphons codes.
    #1	Flat, 2	Peak (summit), 3	Ridge, 4	Shoulder, 5	Spur (convex), 6	Slope, 7	Hollow (concave), 8	Footslope, 9	Valley, 10	Pit (depression)
    wbt.geomorphons(
    file,
    filename,
    search=200,
    threshold=0.0,
    fdist=0,
    skip=0,
    forms=True,
    residuals=False
    )

    move(filename)


def clip_raster_to_polygon(file,filename1,n):
    filename = str(name(file,n))
    filename = (filename + '_tinted_hillshade.tif')

    catchments=[filename1]
    for catchment in catchments:
        wbt.clip_raster_to_polygon(file, f"{catchment}_12m.shp", f"{catchment}_12m.tif", maintain_dimensions=False,)

    #Hypsometric analysis of one or more input digital elevation models (DEMs)
    catchments=[filename1]
    for catchment in catchments:
        wbt.hypsometric_analysis(f"{catchment}.tif", f"{catchment}_12m_hypso.html", watershed=None)

    #hypsometrically tinted shaded relief
    wbt.hypsometrically_tinted_hillshade(
        file,
        filename,
        altitude=45.0,
        hs_weight=0.5,
        brightness=0.5,
        atmospheric=0.0,
        palette="atlas",
        reverse=False,
        zfactor=None,
        full_mode=False,
    )

    move(filename1+'_12m_hypso.html')
    move(filename)


def extract_points(file,filename, DataPuntos, archivo):
    filename = filename + archivo
    ext = len(archivo)

    with open(output + "/" + filename, 'r') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        script_tag = soup.find('script')
        script_text = script_tag.text

        # usar expresión regular para encontrar el diccionario plot
        match = re.search(r'plot\s*=\s*({[^}]+})', script_text)
        if match:
            # extraer la cadena del diccionario y convertirla a un diccionario Python
            plot_text = match.group(1)
            plot_text = re.sub(r'(\w+):', r'"\1":', plot_text)
            plot = json.loads(plot_text)

            # acceder a los datos
            dataX = plot['dataX']
            dataY = plot['dataY']
            puntos = {"X": dataX, "Y": dataY}
            DataPuntos[filename[:-ext] + "X"] = list(dataX[0])
            DataPuntos[filename[:-ext] + "Y"] = list(dataY[0])


def AjusteCurvas(file, filename, puntos):
    x = np.zeros(0)
    y = np.zeros(0)

    for i in puntos:
      if i[-1] == "X":
        x = np.append(x, puntos[i])
      if i[-1] == "Y":
        y = np.append(y, puntos[i])


    fit = np.polyfit(x,y,10)
    print(type(fit), fit)

    x_new = np.linspace(x.min(), x.max(), 100)
    y_new = np.polyval(fit, x_new)


    ###graficar
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

    # Primer subplot: datos originales
    ax1.scatter(x, y, s=1)
    ax1.set_title('Datos')

    # Segundo subplot: polinomio ajustado
    ax2.plot(x_new, y_new, 'r-')
    ax2.set_title('Promedio Curvas')

    plt.title('Promedio de curvas')
    plt.savefig(output[:-6])
    plt.show()



def slope(file, filename):
    filename = filename + '_slope_vs_elevation_plot.html'


    wbt.slope_vs_elevation_plot(
    file,
    filename,
    watershed=None
    )

    move(filename)




def GraficoEdier(data_dir, output, filename, DataPuntos, DataPuntos2, n_grupos):
    periodo = 5

    #puntos grafico de lineas
    x_2 = np.zeros(0)
    y_2 = np.zeros(0)

    for i in DataPuntos2:
      if i[-1] == "X":
        x_2 = np.append(x_2, DataPuntos2[i])
      if i[-1] == "Y":
        y_2 = np.append(y_2, DataPuntos2[i])
    print(len(x_2),len(y_2))


    # Cambiar el tamaño de los arrays a 100 grupos

    tamano_grupo = x_2.size // n_grupos
    x_2 = x_2[:n_grupos*tamano_grupo].reshape((n_grupos, tamano_grupo))
    y_2 = y_2[:n_grupos*tamano_grupo].reshape((n_grupos, tamano_grupo))

    # Calcular el promedio de cada grupo de elementos
    x_2_prom = np.mean(x_2, axis=1)
    y_2_prom = np.mean(y_2, axis=1)


    #grafico barras
    filename2 = data_dir  + '/' + filename
    print(filename)

    with rasterio.open(filename2) as dataset:
        arr = dataset.read(1)


    altitudes = np.arange(0, arr.max()+ n_grupos, n_grupos)        #############################3
    hist, _ = np.histogram(arr, bins=altitudes)
    area_percent = hist / arr.size
    ejex = altitudes[1:]
    ejey = area_percent



    # Crear el subplot con dos ejes y diferentes para cada gráfico
    fig, ax = plt.subplots()

    # Graficar x_2_prom y y_2_prom en un gráfico de barras
    ax.bar(ejex, ejey, color='r', width=n_grupos-1, alpha=0.5)
    ax.set_xlabel('Elevation (m)')
    ax.set_ylabel('Normalized Frequency')
    ax.set_xlim(left=0)


    ax2 = ax.twinx()
    ax2.plot(y_2_prom, x_2_prom)
    ax2.set_ylabel('Mean slope (°)')


    # ax2 = ax1.twinx()  # crear un segundo eje Y
    # ax2.bar(np.arange(len(y_2_prom)), y_2_prom, color='r', alpha=0.5)
    # ax2.set_ylabel('y_2_prom')
    # ax2.tick_params(axis='y', labelcolor='r')

    plt.gcf().canvas.draw()
    img = np.array(plt.gcf().canvas.renderer.buffer_rgba())
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)


    img = output + '/Graf' + filename[:-4] +'.jpg'
    print(img)
    # Mostrar el gráfico
    plt.savefig(img)

    plt.show()










    DataPuntos.clear()
    DataPuntos2.clear()




def crear_carpeta(direccion):
    try:
        os.makedirs(direccion)
        print("Carpeta creada exitosamente en la dirección:", direccion)
    except FileExistsError:
        print("La carpeta ya existe en la dirección:", direccion)
    except Exception as e:
        print("Error al crear la carpeta:", str(e))




In [ ]:
if __name__ == '__main__':
    #definimos dataset
    dataset(data_dir)

    #imprimimos el raster
    #print_raster(file[0],n)

    file1 = file
    print(file1)

    DataPuntos = {}
    DataPuntos2 = {}
    #########################################################
    cont = 0

    # Cargar el archivo de Excel
    df_excel = pd.read_excel('/content/drive/MyDrive/CurvasHipsometricas2/CUENCAS VERSION 2/carpetasTrabajo.xlsx')

    # Crear la variable NomArc
    df_excel['NomArc'] = df_excel['FID'].astype(str) + df_excel['Nombre']

    # Obtener el número de filas
    num_filas = len(df_excel)
    #########################################################

    for i in file1:
      file2 = i
      filename = file2[:-n]
      print(file2)


      ##############################################################################################################
      # Recorrer las filas y mostrar NomArc con contador
      NomArc = df_excel.loc[cont, 'NomArc']
      carpeta = NomArc.replace(' ','')
      cont = cont + 1



      ##################################################################################################################

      output1 = dir
      output = output1 + '/' + str(carpeta)
      crear_carpeta(output)




      ####herramientas de procesamiento individual
      #hillshade(file2,n)
      #slope(file2,n)
      #aspect(file2,n)
      #lan_curvature(file2,n)
      #profile_curvature(file2,n)
      #total_curvature(file2,n)
      #average_normal_vector_angular_deviation(file2,n)  #Roughness
      #roughness_index(file2,n)
      #circular_variance_of_aspect(file2,n)
      #dev_from_mean_elev(file2,n)  #difference between the elevation of each grid cell and the mean elevation of the centering local neighbourhood, normalized by standard deviation
      #diff_from_mean_elev(file2,n) #the difference between the elevation of each grid cell and the mean elevation of the centering local neighbourhood
      #edge_density(file2,n)
      #find_ridges(file2,n)
      geomorphons(file2,n) #geomorphons landform classification. Jasiewicz et al. (2013) identified 10 common landform types by reclassifying related geomorphons codes.
                          #1	Flat, 2	Peak (summit), 3	Ridge, 4	Shoulder, 5	Spur (convex), 6	Slope, 7	Hollow (concave), 8	Footslope, 9	Valley, 10	Pit (depression)
      clip_raster_to_polygon(file2,filename,n)
      extract_points(file2, filename, DataPuntos, "_12m_hypso.html")   #para curvas hypsometricas
      slope(file2, filename)

      extract_points(file2,filename, DataPuntos2, "_slope_vs_elevation_plot.html")   #para slope


      GraficoEdier(data_dir, output,  file2, DataPuntos, DataPuntos2, 5)


    ####herramienta para hacer promedios de todos los archivos / datos
    #AjusteCurvas(file2, filename, DataPuntos)

FileNotFoundError: ignored

In [ ]:


# Cargar el archivo de Excel
df_excel = pd.read_excel('/content/drive/MyDrive/CurvasHipsometricas2/CUENCAS VERSION 2/carpetasTrabajo.xlsx')

# Crear la variable NomArc
df_excel['NomArc'] = df_excel['FID'].astype(str) + df_excel['Nombre']

# Obtener el número de filas
num_filas = len(df_excel)

# Recorrer las filas y mostrar NomArc con contador
for i in range(num_filas):
    NomArc = df_excel.loc[i, 'NomArc']
    print(NomArc.replace(' ',''), i)

0CañadaLaCiénaga 0
1QuebradaSanJuan 1
2QuebradaLaPerica 2
3QuebradaSanJosé 3
4QuebradaSanFrancisco 4
5QuebradaSantaRita 5
6QuebradaAldana 6
7QuebradaSolvetanal 7
8QuebradaGranallillo 8
9QuebradaLaCiquiarca 9
10QuebradaElDiamante 10
11QuebradaSanJoaquín 11
12QuebradaOromira 12
13QuebradaelRecreo 13
14QuebradaElAmparo 14
15QuebradaLaMaría 15
16QuebradaElGranate 16
17QuebradaMontañitas 17
18QuebradaLaSalazar 18
19QuebradaLaCayetana 19
20QuebradaLaZarza 20
21QuebradaSanAntonio 21
22QuebradaPotrerito 22
23QuebradaLaTolda 23
24CañadaPatadeGallina 24
25QuebradaHonda 25
26QuebradaYerbabuena 26
27QuebradaLaPajita 27
28QuebradaRanchería 28
29QuebradadeLaVeta 29
30QuebradaLaPaja 30
31QuebradaElTigre 31
32QuebradadeSantaBarbara 32
33quebradaQuebradita 33
34QuebradaQuebradita 34
35QuebradaLaureles 35
36QuebradaPuentePiedra 36
37QuebradaSeca 37
38QuebradaCampanas 38
39QuebradaLaCabaña 39
40QuebradaCircia 40
41QuebradaLaDoctora 41
42QuebradaLaAyurá 42
43QuebradaDoñaMaría 43
44QuebradaLaGrande 44
45Qu